In [ ]:
from dicom_contour.contour import *
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom
import os
import shutil
import SimpleITK as sitk
import pydicom


In [ ]:
# SRBT Study make directory


path = '/Volumes/External_Drive/PRE_SBRT/'

array_path ='/Volumes/External_Drive/PRE_SBRT_np_array1/'

for root, dirs, files in os.walk(path):
    #if root.endswith('AO/Update') or root.endswith('AO'):
    if not dirs:
        #print(root)

        temp = root.split('/')  # Split root path and remove last subdir
        temp1 = temp[5].split('_')[0]
        temp1 = str(temp1.split(' ')[0]+'_'+temp1.split(' ')[1])

        temp2 = temp[5].split('_')[1]
        print(temp[5])
        temp2 = str(temp2.split(' ')[0]+'_'+temp2.split(' ')[1]+'_'+temp2.split(' ')[2])

        temp3 = str(temp[5].split(' ')[3]+'_')
        new_dir = os.path.join(array_path, temp2)
        print(new_dir)
        # Create Directory
        os.makedirs(new_dir +'/'+temp3+'max_area_image'+'/'+temp1)
        os.makedirs(new_dir +'/'+temp3+'max_area_tumor_contour'+'/'+temp1)
            
            

In [ ]:
# SBRT study convert files into numpy array


path = '/Volumes/External_Drive/PRE_SBRT/'

array_path ='/Volumes/External_Drive/PRE_SBRT_np_array1/'
files_no_contour1 = []
files_no_contour2= []
files_no_contour3 = []
for root, dirs, files in os.walk(path):
    
    if not root.endswith('AO/Update') or root.endswith('AO'):

        if not dirs:
            print(root)
            index1=[]
            
            # Related to the create directory script
            temp = root.split('/')  # Split root path and remove last subdir
            temp1 = temp[5].split('_')[0]
            temp1 = str(temp1.split(' ')[0]+'_'+temp1.split(' ')[1])
            # file label
            file_name = temp1
            
            temp2 = temp[5].split('_')[1]
            temp2 = str(temp2.split(' ')[0]+'_'+temp2.split(' ')[1]+'_'+temp2.split(' ')[2])
            temp3 = str(temp[5].split(' ')[3]+'_')
            # Directory path
            new_dir = os.path.join(array_path, temp2)


            # Numpy array: output image and contour mask
            image, tumor_image_contour = get_data_from_filestore(root)

            m = tumor_image_contour.shape[0]
            #print('shape of tumor image contour',tumor_image_contour.shape)
            if tumor_image_contour.shape[0]>1:
                for k in range(m):
                    temp = np.sum(tumor_image_contour[k,:,:])
                    if temp > 2:

                        # index of slices that have contours
                        index1.append(k)
                        print(index1)
                    else:
                        print('This file has no contour 2',root, tumor_image_contour.shape[0])
                        files_no_contour2.append(root)

                # Extract image and contour slices that only have contours
                print(index1)
                tumor_image_contour1 =tumor_image_contour1[index1,:,:]
                tumor_image_contour = tumor_image_contour[index1,:,:]
                image_with_contour = image[index1,:,:]
                print(tumor_image_contour.shape)
                print(image_with_contour.shape)

                # Fill each contour
                filled_contour = tumor_image_contour
                for k in range(tumor_image_contour.shape[0]):
                    temp1 = binary_dilation(tumor_image_contour[k,:,:],structure=np.ones((5,5)),iterations=1 )
                    cntr = fill_contour(temp1)
                    filled_contour[k,:,:] = cntr
                    plot2dcontour(image_with_contour[k,:,:], tumor_image_contour1[k,:,:])
                    plot2dcontour(image_with_contour[k,:,:], filled_contour[k,:,:])
                # sort contour by area size
                x = np.argsort(-filled_contour.sum(axis = 1).sum(axis = 1))
                print(x)
                # store image and contours in decending area order
                max_area_image = image[x,:,:]
                max_area_tumor_contour = tumor_image_contour1[x,:,:]
                #print(file_name)
                print(str(new_dir +'/'+temp3+'max_area_image'+'/'+file_name))
                np.save( new_dir +'/'+temp3+'max_area_image' +'/'+ file_name, max_area_image)
                np.save( new_dir +'/'+temp3+'max_area_tumor_contour' +'/'+ file_name, max_area_tumor_contour)
                


            else:
                print('This file has no contour 1',root)
                files_no_contour1.append(root)



            # Fill contour mask array
                # loop over contour slices to fill
                #filled_contour_mask_arr = contour_mask_arr
                #if contour_mask_arr.shape[0] > 1:
                #    for i in range(contour_mask_arr.shape[0]):

                #        if np.sum(contour_mask_arr[i,:,:]) > 2:
                #            temp1 = binary_dilation(contour_mask_arr[i,:,:],structure=np.ones((5,5)),iterations=3 )
                            #temp2 = binary_fill_holes(temp1,structure=np.ones((50,50)) )
                #            temp3 = fill_contour(temp1)
                #            plot2dcontour(img_arr[i,:,:], temp1)
                #            plot2dcontour(img_arr[i,:,:], temp3)

                    # Segment out tumor from lung image 
                    #tumor_img_arr = np.where(filled_contour_mask_arr>0, img_arr, 0)

                    #np.save( new_dir +'/'+temp3+'full_img' +'/'+ temp1, img_arr)
                    #np.save( new_dir +'/'+temp3+'tumor_contour' +'/'+ temp1, contour_mask_arr)
